In [4]:
#@title Codigo
!pip install -q dash
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash

# Cargar el DataFrame
df = pd.read_csv('/content/drive/My Drive/datosfinales.csv')  # Carga tu archivo aquí
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df['Fecha Apertura'] = pd.to_datetime(df['Fecha Apertura'], errors='coerce')

# Crear el servidor Dash
app = Dash(__name__)

# Diseño del dashboard
app.layout = html.Div([

    html.H1("Dashboard Interactivo de Datos Financieros", style={'textAlign': 'center', 'color': 'teal', 'fontFamily': 'Arial'}),

    # Filtros
    html.Div([
        html.Label("Seleccionar Tipo de Filtro:"),
        dcc.Dropdown(
            id='tipo-filtro-dropdown',
            options=[
                {'label': 'Nombre', 'value': 'nombre'},
                {'label': 'Combinada', 'value': 'combinada'}
            ],
            value='nombre',  # Valor por defecto
            style={'width': '48%', 'display': 'inline-block'}
        ),
    ], style={'padding': '20px', 'backgroundColor': '#f4f4f9', 'borderRadius': '10px'}),

    # Filtro por Nombre
    html.Div([
        html.Label("Seleccionar Nombre:"),
        dcc.Dropdown(
            id='nombre-dropdown',
            options=[{'label': i, 'value': i} for i in df['Nombre'].unique()],
            value=df['Nombre'].unique()[0],  # Valor por defecto
        ),
    ], id='nombre-container', style={'display': 'block'}),

    # Filtro por Combinada
    html.Div([
        html.Label("Seleccionar Combinada:"),
        dcc.Dropdown(
            id='combinada-dropdown',
            options=[{'label': i, 'value': i} for i in df['Combinada'].unique()],
            value=df['Combinada'].unique()[0],  # Valor por defecto
        ),
    ], id='combinada-container', style={'display': 'none'}),

    # Rango de Fechas
    html.Div([
        html.Label("Rango de Fechas:"),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=df['Fecha'].min(),
            end_date=df['Fecha'].max(),
            display_format='YYYY-MM-DD'
        )
    ], style={'padding': '20px', 'backgroundColor': '#f4f4f9', 'borderRadius': '10px'}),

    # Botón para pantalla completa
    html.Div([
        html.Button("Pantalla Completa", id="fullscreen-button", n_clicks=0, style={'fontSize': '20px', 'backgroundColor': '#4CAF50', 'color': 'white', 'padding': '10px 20px', 'border': 'none', 'borderRadius': '5px'}),
    ], style={'padding': '20px', 'textAlign': 'center'}),

    # Gráficos
    dcc.Graph(id="grafico_saldo", style={'padding': '20px'}),
    dcc.Graph(id="grafico_rentabilidad", style={'padding': '20px'}),
    dcc.Graph(id="grafico_variacion_unidades", style={'padding': '20px'}),
    dcc.Graph(id="grafico_valor_fondo", style={'padding': '20px'}),

    # Información adicional
    html.Div([
        html.Div([
            html.H3('Número de Cuentas Seleccionadas'),
            html.Div(id='numero_cuentas', style={'fontSize': '24px', 'color': 'teal', 'fontWeight': 'bold'})
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            html.H3('Porcentaje Total'),
            html.Div(id='porcentaje_total', style={'fontSize': '24px', 'color': 'teal', 'fontWeight': 'bold'})
        ], style={'width': '48%', 'display': 'inline-block'})
    ], style={'padding': '20px', 'backgroundColor': '#e0f7fa', 'borderRadius': '10px'})
])

# Callback para mostrar solo el filtro correspondiente
@app.callback(
    [Output('nombre-container', 'style'),
     Output('combinada-container', 'style')],
    [Input('tipo-filtro-dropdown', 'value')]
)
def toggle_filters(filtro_seleccionado):
    if filtro_seleccionado == 'nombre':
        return {'display': 'block'}, {'display': 'none'}
    else:
        return {'display': 'none'}, {'display': 'block'}

# Callback para actualizar los gráficos y la información
@app.callback(
    [Output("grafico_saldo", "figure"),
     Output("grafico_rentabilidad", "figure"),
     Output("grafico_variacion_unidades", "figure"),
     Output("grafico_valor_fondo", "figure"),
     Output('numero_cuentas', 'children'),
     Output('porcentaje_total', 'children')],
    [Input("nombre-dropdown", "value"),
     Input("combinada-dropdown", "value"),
     Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date"),
     Input("tipo-filtro-dropdown", "value")]
)
def update_graph(selected_nombre, selected_combinada, start_date, end_date, filtro_seleccionado):
    # Filtrar el DataFrame basado en los inputs
    if filtro_seleccionado == 'nombre':
        filtered_df = df[
            (df['Nombre'] == selected_nombre) &
            (df['Fecha'] >= start_date) &
            (df['Fecha'] <= end_date)
        ]
        color_column = 'Nombre'
    else:
        filtered_df = df[
            (df['Combinada'] == selected_combinada) &
            (df['Fecha'] >= start_date) &
            (df['Fecha'] <= end_date)
        ]
        color_column = 'Combinada'

    filtered_df = filtered_df.groupby(['Fecha', color_column], as_index=False).agg({
        'Saldo': 'sum',
        'Valor del fondo': 'sum',
        'Rent. Neta dia': 'first',
        'Variacion Unidades': 'first'
    })

    # Número de cuentas y porcentaje (solo si se filtra por Nombre)
    if filtro_seleccionado == 'nombre':
        num_cuentas = len(filtered_df)
        total_saldo = filtered_df['Saldo'].sum()
        total_saldo_global = df[df['Fecha'] <= end_date]['Saldo'].sum()
        porcentaje_total = (total_saldo / total_saldo_global) * 100
        cuentas_text = f'{num_cuentas} cuentas con "{selected_nombre}"'
        porcentaje_text = f'{porcentaje_total:.2f}% del saldo total'
    else:
        cuentas_text = 'N/A'
        porcentaje_text = 'N/A'

    # Gráfico 1: Saldo
    fig_saldo = go.Figure(data=[go.Bar(x=filtered_df['Fecha'], y=filtered_df['Saldo'], name=f"Saldo ({selected_nombre if filtro_seleccionado == 'nombre' else selected_combinada})", marker_color='rgba(55, 128, 191, 0.7)')])
    fig_saldo.update_layout(
        title="Saldo a través del tiempo",
        xaxis_title="Fecha",
        yaxis_title="Saldo",
        template="plotly_dark",
        showlegend=True
    )

    # Gráfico 2: Rentabilidad
    fig_rentabilidad = go.Figure(data=[go.Scatter(x=filtered_df['Fecha'], y=filtered_df['Rent. Neta dia'], mode='lines+markers', name=f"Rentabilidad ({selected_nombre if filtro_seleccionado == 'nombre' else selected_combinada})", line=dict(color='orange', width=3))])
    fig_rentabilidad.update_layout(
        title="Rentabilidad Neta Diaria",
        xaxis_title="Fecha",
        yaxis_title="Rentabilidad (%)",
        template="plotly_dark",
        showlegend=True
    )

    # Gráfico 3: Variación de Unidades
    fig_variacion = go.Figure(data=[go.Scatter(x=filtered_df['Fecha'], y=filtered_df['Variacion Unidades'], mode='lines+markers', name=f"Variación Unidades ({selected_nombre if filtro_seleccionado == 'nombre' else selected_combinada})", line=dict(color='green', width=3))])
    fig_variacion.update_layout(
        title="Variación de Unidades",
        xaxis_title="Fecha",
        yaxis_title="Variación de Unidades",
        template="plotly_dark",
        showlegend=True
    )

    # Gráfico 4: Valor del Fondo
    fig_valor_fondo = go.Figure(data=[go.Bar(x=filtered_df['Fecha'], y=filtered_df['Valor del fondo'], name=f"Valor del Fondo ({selected_nombre if filtro_seleccionado == 'nombre' else selected_combinada})", marker_color='rgba(255, 165, 0, 0.7)')])
    fig_valor_fondo.update_layout(
        title="Valor del Fondo a través del tiempo",
        xaxis_title="Fecha",
        yaxis_title="Valor del Fondo",
        template="plotly_dark",
        showlegend=True
    )

    return fig_saldo, fig_rentabilidad, fig_variacion, fig_valor_fondo, cuentas_text, porcentaje_text


# Callback para pantalla completa
@app.callback(
    Output("grafico_saldo", "style"),
    Input("fullscreen-button", "n_clicks")
)
def toggle_fullscreen(n_clicks):
    if n_clicks > 0:
        return {'width': '100%', 'height': '100vh'}
    return {'padding': '20px'}

# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server(debug=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-ce00c6ccb7b9>:12: DtypeWarning:

Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.



<IPython.core.display.Javascript object>

In [5]:
df.columns

Index(['Combinada', 'Descripcion', 'Descripcion de Tp. Participacion',
       'Empresa Encargo', 'Estado Encargo', 'Fecha', 'Fecha Apertura',
       'Nombre', 'No. Unidades', 'Oficina', 'Rent. Neta dia', 'Saldo',
       'Tipo de Participacion', 'Tipo de inversion', 'Valor de la unidad',
       'Valor del fondo', 'Variacion Saldos', 'Descripcion del tp. Inv.',
       'Identificacion Titular', 'Nombre Titular', 'Numero Encargo',
       'Variacion Unidades'],
      dtype='object')